<a href="https://colab.research.google.com/github/columbia-data-club/meetings/blob/main/2022/september-29-holoviz-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">
Holoviz: Data visualization made easy (and cool!)

Columbia Data Club   
September 29, 2022  
Roger Creel  
</h3>

 -----------------------------------


# Source Attribution
This notebook is a composite, with tweaks and significant text edits, of tutorials from the [Holoviz Tutorial Library](https://holoviz.org/tutorial/index.html)

 -----------------------------------


In [ ]:
!pip install -q fastparquet pyviz datashader
!pip install -U bokeh holoviews git+https://github.com/holoviz/pyviz_comms


In [ ]:
import os
import pandas as pd
import calendar
import datetime as dt
import requests
import pathlib
import numpy as np

!mkdir ./data
!mkdir ./data/years/
!mkdir ./output/

# Load Earthquake data

Our first step is to download the USGS catalogue of 21st century earthquakes.  It takes a few minutes. 

In [ ]:
URL = "https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime={start}&endtime={end}&minmagnitude=2.0&orderby=time"

for yr in range(2000, 2023):
    print(yr)
    for m in range(1, 13):
        if os.path.isfile('data/years/{yr}_{m}.csv'.format(yr=yr, m=m)):
            continue
        _, ed = calendar.monthrange(yr, m)
        start = dt.datetime(yr, m, 1)
        end = dt.datetime(yr, m, ed, 23, 59, 59)
        with open('data/years/{yr}_{m}.csv'.format(yr=yr, m=m), 'w', encoding='utf-8') as f:
            f.write(requests.get(URL.format(start=start, end=end)).content.decode('utf-8'))

dfs = []
for i in range(2000, 2023):
    for m in range(1, 13):
        if not os.path.isfile('./data/years/%d_%d.csv' % (i, m)):
            continue
        df = pd.read_csv('./data/years/%d_%d.csv' % (i, m), dtype={'nst': 'float64'})
        dfs.append(df)
df = pd.concat(dfs, sort=True)
df.to_parquet('data/earthquakes.parq', 'fastparquet')

# Plotting with Holoviz

There are many ways to represent data, including data tables, textual summaries and so on. We'll focus on plotting data using a simple but powerful plotting API.

If you have tried to visualize a `pandas.DataFrame` before, then you have likely encountered the [Pandas .plot() API](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html). These plotting commands use [Matplotlib](http://matplotlib.org) to render static PNGs or SVGs in a Jupyter notebook using the `inline` backend, or interactive figures via `%matplotlib widget`, with a command as simple as `df.plot()` for a DataFrame with 1-2 columns. 

The Pandas .plot() API is a standard for high-level plotting in Python, and is now supported by many different libraries that use various plotting engines. Learning this API allows you to access a variety of underlying tools with little additional effort, including:


- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) -- Matplotlib-based API included with Pandas.
- [xarray](https://xarray.pydata.org/en/stable/plotting.html) -- Matplotlib-based API included with xarray, based on pandas .plot API. 
- [hvPlot](https://hvplot.pyviz.org) -- Bokeh/Matplotlib/Plotly-based HoloViews plots for Pandas, GeoPandas, xarray, Dask, Intake, and Streamz data.
- [Pandas Bokeh](https://github.com/PatrikHlobil/Pandas-Bokeh) -- Bokeh-based interactive plots, for Pandas, GeoPandas, and PySpark data.
- [Cufflinks](https://github.com/santosjorge/cufflinks) -- Plotly-based interactive plots for Pandas data.
- [Plotly Express](https://plotly.com/python/pandas-backend) -- Plotly-Express-based interactive plots for Pandas data; only partial support for the .plot API keywords.
- [PdVega](https://altair-viz.github.io/pdvega) -- Vega-lite-based, JSON-encoded interactive plots for Pandas data.

Here we'll explore the default `.plot` API and demonstrate the capabilities provided by `.hvplot`, which include interactivity in notebooks and deployed dashboards, server-side rendering of large datasets, automatic small multiples and widget selectors for exploring complex data, and easy composition and linking of plots. 

To show these features, we'll use a tabular dataset of earthquakes queried 
from the [USGS Earthquake Catalog](https://earthquake.usgs.gov/earthquakes/search) using its 
[API](https://github.com/pyviz/holoviz/wiki/Creating-the-USGS-Earthquake-dataset). The approach used here can be used with any tabular dataset, and similar approaches can be used with [gridded (multidimensional array) datasets](https://hvplot.holoviz.org/user_guide/Gridded_Data.html).


## Read in the data


In [ ]:
columns = ['depth', 'id', 'latitude', 'longitude', 'mag', 'place', 'time', 'type']
path = pathlib.Path('./data/earthquakes.parq')
df = pd.read_parquet(path, columns=columns, engine='fastparquet')
df.head()

In [ ]:
%%time
df = df.set_index(df.time)
print(df.shape)
df.head()

To compare HoloViz approaches with other approaches, we'll also subsample 1% of the dataset so it is tractable with any plotting or analysis tool:

In [ ]:
small_df = df.sample(frac=.01)
print(small_df.shape)
small_df

## Using Pandas `.plot()`

Let's first visualize every earthquake location by making a scatter or points plot where _x_ is longitude and _y_ is latitude using the `pandas.plot` API and Matplotlib:


In [ ]:
%matplotlib inline

small_df.plot.scatter(x='longitude', y='latitude');

## Using `.hvplot`
The Pandas API easily gives you a usable plot, where you can start to see the outlines of the tectonic plates, which in many cases correspond with the visual edges of continents (e.g. the westward side of Africa, in the center). You can make a similar plot with the same arguments using hvplot, after importing `hvplot.pandas` to install hvPlot support into Pandas.  

There is one other thing that needs explanation. 

* We're including the `hv.extension('bokeh', logo=False)` line at the beginning of every cell. This lets Holoviews know to display the elements with Bokeh (but to not show the Bokeh logo every time in the output). If you are running your script locally (not on Google Colab), you only need this once. Supposedly it is also possible to make the Colab notebook remember it without repeating, but that proved unreliable for me.   



In [ ]:
import hvplot.pandas # noqa: adds hvplot method to pandas objects
import holoviews as hv
# from bokeh.plotting import gridplot, show

In [ ]:
hv.extension('bokeh', logo=False)

small_df.hvplot.scatter(x='longitude', y='latitude')

Here unlike in the Pandas `.plot()` the displayed plot is a **Bokeh** plot that has a default hover action on the datapoints to show location values, and you can pan and zoom to focus on any region of interest. 

You might have noticed that many of the dots in the scatter that we've just created cover each other. This is called ["overplotting"](https://datashader.org/user_guide/Plotting_Pitfalls.html) and can be avoided in a variety of ways, such as by making the dots slightly transparent, or binning the data. 


### Exercise

Try changing the alpha (try .1) on the plot above to see the effect of this approach
<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python
small_df.hvplot.scatter(x='longitude', y='latitude', alpha=0.1)
```
</details>

Try creating a `hexbin` plot.
<details><summary><i><u>(Solution)</u><i></summary><br>

```python
small_df.hvplot.hexbin(x='longitude', y='latitude')
```
</details>

You may wonder how you could have discovered the `alpha` keyword option or other `hvplot` options. For this purpose, use tab-completion in the Jupyter notebook or the `hvplot.help` function which are documented in the [user guide](https://hvplot.holoviz.org/user_guide/Customization.html).

For tab completion, you can press tab after the opening parenthesis in a `obj.hvplot.<kind>(` call. For instance, try pressing tab after the partial expression `small_df.hvplot.scatter(<TAB>`.

Alternatively, call `hvplot.help(<kind>)` to see a documentation pane pop up in the notebook. Try uncommenting the following line and executing it:


In [ ]:
# hvplot.help('scatter')


You will see there are a lot of options!  You can control which section of the documentation you view with the `generic`, `docstring` and `style` boolean switches also documented in the  [user guide](https://hvplot.holoviz.org/user_guide/Customization.html). If you run the following cell, you will see that `alpha` is listed in the 'Style options'.


In [ ]:
# hvplot.help('scatter', style=True, generic=False)


These style options refer to options that are part of the Bokeh API. This means that the `alpha` keyword is passed directly to Bokeh like the other style options. Find out more by using the search functionality in the [Bokeh docs](https://docs.bokeh.org/en/latest/).


## Datashader

We often face arbitrary choices before we understand a dataset's properties, such as selecting an alpha value or a bin size for aggregation. Making such assumptions can bias you, and having to throw away 99% of the data can cover up otherwise-clear patterns. To explore a new dataset, it's safer if you can just ***see*** the data before you assume form or structure, and without having to subsample.

To avoid some problems of traditional scatter/point plots we can use hvPlot's [Datashader](datashader.org) support. Datashader aggregates data into each pixel without arbitrary settings, making data visible immediately. In `hvplot` we can activate this capability by setting `rasterize=True` to invoke Datashader before rendering and `cnorm='eq_hist'` (["histogram equalization"](https://datashader.org/user_guide/Plotting_Pitfalls.html)) to specify that the colormapping should adapt to data distribution:


In [ ]:
hv.extension('bokeh', logo=False)

small_df.hvplot.scatter(x='longitude', y='latitude', rasterize=True, cnorm='eq_hist')


We can already see more detail, but remember that we are still only plotting 1% of the data. With Datashader, we can quickly and easily plot all of the full, original dataset of 2.1 million earthquakes:


In [ ]:
hv.extension('bokeh', logo=False)

df.hvplot.scatter(x='longitude', y='latitude', rasterize=True, cnorm='eq_hist', dynspread=True)


Here you can see the full set of earthquake event locations. You can zoom in and see more detail at each zoom level without tuning parameters or making assumptions about data form or structure. You can also specify colormapping `cnorm='log'` or the default `cnorm='linear'`, which are easier to interpret, but starting with `cnorm='eq_hist'` is a good idea so you can see the shape of the data before committing to an easier-to-interpret but potentially data-obscuring colormap. Learn more about Datashader [here](https://holoviews.org/user_guide/Large_Data.html). For now, the important thing is that with Datashader we can work with arbitrarily large datasets in a web browser conveniently.

Here we used  `.hvplot()` on a Pandas dataframe, but (unlike other `.plot` libraries), the same commands will work on many other libraries after the appropriate import (`import hvplot.xarray`, `import hvplot.dask`, etc.): 
 - Pandas : DataFrame, Series (columnar/tabular data)
 - xarray : Dataset, DataArray (labelled multidimensional arrays)
 - Dask : DataFrame, Series (distributed/out of core arrays and columnar data)
 - Streamz : DataFrame(s), Series(s) (streaming columnar data)
 - Intake : DataSource (data catalogues)
 - GeoPandas : GeoDataFrame (geometry data)
 - NetworkX : Graph (network graphs)


#### Exercise


Select a subset of the data, e.g. only magitudes >5 and plot them with a different colormap (valid `cmap` values include 'viridis_r', 'Reds' and 'magma_r'):
<details><summary><i><u>(Solution)</u><i></summary><br>

```python
df[df.mag>5].hvplot.scatter(x='longitude', y='latitude', rasterize=True, cnorm='eq_hist', cmap='Reds')
```
    
</details>

## Statistical Plots

Let's dive into other capabilities of `.plot()` and `.hvplot()`, starting with the frequency of different magnitude earthquakes.

| Magnitude     | Earthquake Effect | Estimated Number Each Year |
|---------------|-------------------|----------------------------|
| 2.5 or less   | Usually not felt, but can be recorded by seismograph. |900,000|
| 2.5 to 5.4    | Often felt, but only causes minor damage. |30,000 |
| 5.5 to 6.0    | Slight damage to buildings and other structures. |500 |
| 6.1 to 6.9    | May cause a lot of damage in very populated areas. | 100 |
| 7.0 to 7.9    | Major earthquake. Serious damage. | 20 |
| 8.0 or greater| Great earthquake. Can totally destroy communities near the epicenter. | One every 5 to 10 years |

We'll use a histogram first with `.plot.hist`, then with `.hvplot.hist`. Before plotting we clean the data by setting magnitudes less than 0 to NaN.


In [ ]:
cleaned_df = df.copy()
cleaned_df['mag'] = df.mag.where(df.mag > 0)


In [ ]:
cleaned_df.plot.hist(y='mag', bins=50);

In [ ]:
hv.extension('bokeh', logo=False)

df.hvplot.hist(y='mag', bin_range=(0, 10), bins=50)


#### Exercise

Create a kernel density estimate (kde) plot of magnitude for `cleaned_df`:
<details><summary><i><u>(Solution)</u><i></summary><br>

```python
cleaned_df.hvplot.kde(y='mag')
```

</details>

## Categorical variables

Next we'll categorize the earthquakes based on depth. You can read about all the different variables available in this dataset [here](https://earthquake.usgs.gov/data/comcat/data-eventterms.php). According to the [USGS page on earthquake depths](https://www.usgs.gov/natural-hazards/earthquake-hazards/science/determining-depth-earthquake?qt-science_center_objects=0#qt-science_center_objects), typical depth categories are:

| Depth class   | Depth       | 
|---------------|-------------|
| shallow       | 0   -  70 km| 
| intermediate  | 70  - 300 km| 
| deep          | 300 - 700 km| 

First we'll use `pd.cut` to split the small_dataset into depth categories.


In [ ]:
depth_bins = [-np.inf, 70, 300, np.inf]
depth_names = ['Shallow', 'Intermediate', 'Deep']
depth_class_column = pd.cut(cleaned_df['depth'], depth_bins, labels=depth_names)


In [ ]:
cleaned_df.insert(1, 'depth_class', depth_class_column)


We can now use this new categorical variable to group our data. First we will overlay all our groups on the same plot using the `by` option:


In [ ]:
hv.extension('bokeh', logo=False)

cleaned_df.hvplot.hist(y='mag', by='depth_class', alpha=0.6)


**NOTE:** Click on the legend to turn off certain categories and see what is behind them.


Add `subplots=True` and `width=300` to see the different classes side-by-side instead of overlaid. The axes will be linked, so try zooming.


## Grouping
What if you want a single plot, but want to see each class separately? Use the `groupby` option to get a widget for toggling between classes, here in a bivariate plot (using a subset of the data as bivariate plots can be expensive to compute):


In [ ]:
hv.extension('bokeh', logo=False)

cleaned_small_df = cleaned_df.sample(frac=.01)
cleaned_small_df.hvplot.bivariate(x='mag', y='depth', groupby='depth_class')


In addition to classifying by depth, we can classify by magnitude.

| Magnitude Class| Magnitude | 
|----------------|-----------|
| Great          | 8 or more | 
| Major          | 7 - 7.9   | 
| Strong         | 6 - 6.9   |
| Moderate       | 5 - 5.9   |
| Light          | 4 - 4.9   |
| Minor          | 3 -3.9    |


In [ ]:
classified_df = df[df.mag >= 3].copy()

depth_class = pd.cut(classified_df.depth, depth_bins, labels=depth_names)

classified_df['depth_class'] = depth_class

mag_bins = [2.9, 3.9, 4.9, 5.9, 6.9, 7.9, 10]
mag_names = ['Minor', 'Light', 'Moderate', 'Strong', 'Major', 'Great']
mag_class = pd.cut(classified_df.mag, mag_bins, labels=mag_names)
classified_df['mag_class'] = mag_class

categorical_df = classified_df.groupby(['mag_class', 'depth_class']).count()


With data binned in two categories, we can use a logarithmic heatmap to visually represent this data as the count of detected earthquake events in each combination of depth and mag class:


In [ ]:
hv.extension('bokeh', logo=False)

categorical_df.hvplot.heatmap(x='mag_class', y='depth_class', C='id',
                              logz=True, clim=(1, np.nan))

## Output Matplotlib or Plotly plots

While the default plotting backend/extension of hvPlot is Bokeh, it also supports rendering plots with either [Plotly](https://plotly.com/python/) or [Matplotlib](https://matplotlib.org/). Like Bokeh, Plotly is an interactive library that provides similar data-exploring actions (pan, hover, zoom, etc.). You can decide to use Plotly instead of Bokeh if you prefer its look and feel, need interactive 3D plots, or if your organization's policy is to use that plotting library. Matplotlib is supported for static plots only (e.g. PNG or SVG), which is useful for saving and sharing images and embedding them in documents for publication.

To load a plotting backend you can use the `hvplot.extension` function. The first backend you declare in the call to the function will be set as the default one.


In [ ]:
hvplot.extension('plotly', 'matplotlib')

small_df.hvplot.scatter(x='longitude', y='latitude')

Once a backend is loaded with `hvplot.extension` you can use the `hvplot.output` function to switch from one backend to another.


In [ ]:
hvplot.output(backend='matplotlib')

plot = small_df.hvplot.scatter(x='longitude', y='latitude')
plot


## Save and further customize plots

You can easily save a plot with the `hvplot.save` function in one of the output formats supported by the backend. This is particularly useful when you are using Matplotlib as your plotting backend, with which you can create static plots suited for publication.


In [ ]:
fp_png, fp_svg = './output/plot.png', './output/plot.svg'
hvplot.save(plot, fp_png)
hvplot.save(plot, fp_svg)


In [ ]:
from IPython.display import display, Image, SVG

display(Image(fp_png))
display(SVG(fp_svg))


While hvPlot allows you to customize your plots quite extensively there are always situations where you want to customize them a little more than what can be done directly with hvPlot. In those cases you can use the `hvplot.render` function to get a handle on the underlying figure object. Below we're using Matplotlib's `xkcd` context manager to turn on *xkcd* sketch-style drawing mode while rendering the figure.


In [ ]:
import matplotlib.pyplot as plt

with plt.xkcd():
    mpl_fig = hvplot.render(plot)

print(mpl_fig)
mpl_fig

`mpl_fig` is a Matplotlib `Figure` object that we could further customize using [Matplotlib's API](https://matplotlib.org/stable/api).


As you can see, hvPlot makes it simple to explore your data interactively, with commands based on the widely used Pandas `.plot()` API but now supporting many more features and different types of data. The following section will focus on how to assemble these plots once you have them, linking them to understand and show their structure.  We'll examine the output of hvPlot calls to take a look at individual HoloViews objects. Then we will see how these \"elements\" offer us powerful ways of combining and composing layered visualizations.


We'll reindex by time so that we can resample the data to explore patterns in number and magnitude of earthquakes over time.:

In [ ]:
df.index = pd.to_datetime(df.index)

weekly_count = df.id.resample('1W').count().rename('count')
weekly_count_plot = weekly_count.hvplot(title='weekly count')

First, note that with `hvplot`, it is common to grab a handle on the returned output. The value returned from the Matplotlib based `.plot` API of Pandas is an axis object that is typically discarded, with plotting display occurring only as a side effect if `%matplotlib inline` is loaded. With hvPlot, there are no side effects; the plot is displayed only if it is returned as the last value in the Jupyter cell (and thus no plot is visible in the above cell's output). hvPlot objects thus work like any other normal Python object; just as you don't expect `x=2` to display anything, `x=df.hvplot()` will not display anything; both simply assign a value to `x`.

Once you have a handle, however, you can plot it if you wish:

In [ ]:
hv.extension('bokeh', logo=False)

weekly_count_plot

But we can also do other things, such as look at its textual representation by printing it:

In [ ]:
print(weekly_count_plot)

Now let’s do a similar resampling, but for magnitude:



`:Curve   [time]   (count)`\" in HoloViews notation means that this object is a `Curve` element with `time` as a key dimension (`kdim`, in square brackets) and `count` as a value dimension (`vdim`, in parentheses). In other contexts, key dimensions are also called index dimensions or independent variables, while value dimensions are also called dependent variables. 

Now let's do a similar resampling, but for magnitude:

In [ ]:
hv.extension('bokeh', logo=False)

weekly_mean_magnitude = df.mag.resample('1W').mean()
weekly_mean_magnitude_plot = weekly_mean_magnitude.hvplot(title='weekly mean magnitude')
weekly_mean_magnitude_plot

In [ ]:
print(weekly_mean_magnitude_plot)

This plot has time on the x axis like the other, but the value dimension is magnitude rather than count. HoloViews objects can be composed into an overlay using a `*` symbol, with a legend generated to distinguish them:

In [ ]:
hv.extension('bokeh', logo=False)

weekly_mean_magnitude_plot * weekly_count_plot

The two timeseries have quite different value ranges, making it hard to see the fluctuations in magnitude. A more useful form of composition here is a layout of separate plots, using a `+` symbol to compose HoloViews objects side-by-side with axes linked for any shared dimensions:

In [ ]:
hv.extension('bokeh', logo=False)

(weekly_mean_magnitude_plot + weekly_count_plot).cols(1)


# Adding a Third Dimension

Now let's filter the earthquakes to only include the really high intensity ones. Using the pandas `.plot()` API, we can add extra dimensions to the visualization by using color to represent magnitude in addition to the x and y locations:

In [ ]:
from holoviews.util.transform import lon_lat_to_easting_northing
df.loc[:, 'easting'], df.loc[:, 'northing'] = lon_lat_to_easting_northing(df.longitude,df.latitude)

most_severe = df[df.mag >= 7]


%matplotlib inline
most_severe.plot.scatter(x='longitude', y='latitude', c='mag');

Here is the analogous version using `hvplot` where we grab the handle `high_mag_scatter` so we can inspect the return value:

In [ ]:
hv.extension('bokeh', logo=False)

high_mag_scatter = most_severe.hvplot.scatter(x='longitude', y='latitude', c='mag')
high_mag_scatter

As always, this return value is actually a HoloViews element which has a printed representation:

In [ ]:
print(high_mag_scatter)

This representation reveals that even though the scatterplot here looks ok, it's actually flawed. Earthquakes occur at a particular 2D location on the earth's surface and have a measured magnitude, i.e., they are 2D points with some associated value, and should have a representation like [longitude,latitude] (mag).  There should be two key dimensions (independent variables), with the magnitude being a dependent variable ("value dimension"). The problem here is that the Pandas .scatter call does not distinguish between these types of dimensions, which will confuse HoloViews when it is doing automatic linking and other operations that depend on the interpretation of the data. For this purpose, HoloViews provides a separate `.hvplot.points` call that has the same visual representation but the correct semantics:

In [ ]:
hv.extension('bokeh', logo=False)

high_mag_points = most_severe.hvplot.points(x='longitude', y='latitude', c='mag')
high_mag_points

This object now appropriately represents that latitude and longitude are the key dimensions, with one value dimension (the magnitude).

Let's adjust the options to create a better plot. First we'll use [colorcet](https://colorcet.pyviz.org) to get a colormap that will have a good contrast with blue oceans when we show earthquakes on a map; the default blue colormap above would get lost against the seas! We can choose one from the website and use the HoloViews/Bokeh-based `colorcet` plotting module to make sure it looks good.

In [ ]:
hv.extension('bokeh', logo=False)

import colorcet as cc
from colorcet.plotting import swatch

swatch('CET_L4')

We'll reverse the colors to align dark reds with higher magnitude earthquakes for better contrast.

In [ ]:
hv.extension('bokeh', logo=False)

mag_cmap = cc.CET_L4[::-1]
swatch("CET_L4_r", mag_cmap)

In addition to fixing the colormap, we will add a title and some additional columns to the hover text.

In [ ]:
hv.extension('bokeh', logo=False)

high_mag_points = most_severe.hvplot.points(
    x='longitude', y='latitude', c='mag', hover_cols=['place', 'time'],
    cmap=mag_cmap,  title='Earthquakes with magnitude >= 7')

high_mag_points

# Overlay with Tiled Map

The \"CET_L4\" colormap works well here, and we can kind of see the outlines of the continents, but the visualization would be much easier to parse if we added a base map underneath. To do this, we'll import a tile element from HoloViews, namely the `EsriImagery` tiles from [ESRI](https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9) using the Web Mercator projection:

In [ ]:
hv.extension('bokeh', logo=False)

from holoviews.element.tiles import EsriImagery
EsriImagery()

In order to overlay on this basemap, we will  project our earthquakes to the Web Mercator projection system used by Bokeh. To do that use the Web Mercator `easting` (meters East of Greenwich) and `northing` (meters north of the equator) columns that we calculated from `longitude` and `latitude` earlier using the HoloViews [hv.util.transform.lon_lat_to_easting_northing](https://holoviews.org/reference_manual/holoviews.util.html#holoviews.util.transform.lon_lat_to_easting_northing) function, then overlay our points on top of the `EsriImagery` tile source by using the HoloViews `*` operator:

In [ ]:
hv.extension('bokeh', logo=False)

EsriImagery() * most_severe.hvplot.points(
    x='easting', y='northing', c='mag', hover_cols=['place', 'time'], 
    cmap=mag_cmap, title='Earthquakes with magnitude >= 7', line_color='black')

Actually, for this special but common case of overlaying data on geographic tiles, hvPlot lets you simply specify `tiles='EsriImagery'` as a string in the `hvplot.points` call instead of explicitly overlaying a tile source with `*`

In [ ]:
hv.extension('bokeh', logo=False)

most_severe.hvplot.points(
    x='easting', y='northing', c='mag', hover_cols=['place', 'time'], 
    cmap=mag_cmap, title='Earthquakes with magnitude >= 7', tiles='EsriImagery',
    line_color='black')

<h3 align="center"> That's all for now folks.

  Come back in two weeks for even cooler plotting interactivity! </h3>